# Full Task COVID-19 Tracker Provinsi Riau
Pada notebook ini, saya merangkum semua task yang dijalankan pada repository ini. Tujuan saya membuat project ini karena keresahan saya dimana tidak ada histori data yang tersimpan di website https://covid19.riau.go.id/, dan yang ditampilkan hanya data harian. Oleh karena itu, saya ingin membuat semacam script dengan output seperti berikut,

- Melakukan Scraping terhadap website https://covid19.riau.go.id/
- Membuat dan mengupdate dataset (Baik untuk provinsi maupun daerah) per harinya
- Membuat dashboard yang menampilkan grafik perkembangan COVID-19 per daerah
- Membuat chatbot yang menampilkan rangkuman dari COVID-19 per harinya

Untuk teknologi yang digunakan, saya menggunakan

- Python dan R sebagai bahasa pemrograman
- Library requests dan bs4 untuk ekstraksi informasi dari website
- Shiny untuk pengembangan web app dashboard COVID-19

In [1]:
import csv
import re
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup

## Update Kasus COVID-19 (Keseluruhan)

In [2]:
r = requests.get('https://covid19.riau.go.id/pantauan_data_kasus')
soup = BeautifulSoup(r.content, 'html.parser')

### Untuk Ditampilkan Di Chatbot

In [4]:
for kota in soup.tbody.find_all('tr'):
    data = kota.find_all('td')
    print(data[0].text)
    print("Kasus Aktif", data[-3].text)
    print("Sembuh", data[-2].text)
    print("Meninggal", data[-1].text)
    print()

Pekanbaru
Kasus Aktif 37
Sembuh 106
Meninggal 6

Dumai
Kasus Aktif 7
Sembuh 28
Meninggal 0

Bengkalis
Kasus Aktif 1
Sembuh 19
Meninggal 1

Siak
Kasus Aktif 32
Sembuh 11
Meninggal 0

Meranti
Kasus Aktif 1
Sembuh 12
Meninggal 0

Pelalawan
Kasus Aktif 1
Sembuh 12
Meninggal 0

Indragiri Hilir
Kasus Aktif 6
Sembuh 45
Meninggal 2

Indragiri Hulu
Kasus Aktif 0
Sembuh 3
Meninggal 1

Kuantan Singingi
Kasus Aktif 1
Sembuh 11
Meninggal 0

Kampar
Kasus Aktif 14
Sembuh 14
Meninggal 1

Rokan Hulu
Kasus Aktif 4
Sembuh 5
Meninggal 0

Rokan Hilir
Kasus Aktif 1
Sembuh 0
Meninggal 0



### Dijadikan Dataset (Update Per Hari)
#### Code:
```{py}
with open('regional-dataset.csv', 'a+') as csv_file:
    fieldnames = ['tanggal', 'kabupaten/kota', 'odp_proses_pemantauan', 'odp_selesai_pemantauan', 'pdp_masih_dirawat', 'pdp_pulang_sehat', 'pdp_meninggal', 'positif_masih_dirawat', 'positif_pulang_sehat', 'positif_meninggal']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    for kota in soup.tbody.find_all('tr'):
        temp = {}
        c = 1
        temp['tanggal'] = date.today()
        for data in kota.find_all('td'):
            data = data.text
            data = re.sub("\.", "", data)
            temp[fieldnames[c]] = data
            c += 1
        writer.writerow(temp)  
```

In [20]:
df = pd.read_csv('regional-dataset.csv')
df.tail(12)

,Tanggal,Wilayah,(ODP) Sedang Pemantauan,(ODP) Selesai Pemantauan,(PDP) Sedang Dirawat,(PDP) Sembuh,(PDP) Meninggal,(Positif) Sedang Dirawat,(Positif) Sembuh,(Positif) Meninggal
0,2020-07-28,Pekanbaru,303,6626,82,967,87,37,106,6
1,2020-07-28,Dumai,363,3238,9,160,13,7,28,0
2,2020-07-28,Bengkalis,78,7676,2,160,18,1,19,1
3,2020-07-28,Siak,327,5073,37,108,10,32,11,0
4,2020-07-28,Meranti,735,9516,2,33,1,1,12,0
5,2020-07-28,Pelalawan,33,3675,3,107,13,1,12,0
6,2020-07-28,Indragiri Hilir,340,13119,10,243,10,6,45,2
7,2020-07-28,Indragiri Hulu,1,660,3,35,2,0,3,1
8,2020-07-28,Kuantan Singingi,309,8180,6,53,10,1,11,0
9,2020-07-28,Kampar,170,6732,25,257,34,14,14,1


## Update Kasus COVID-19 (Per Kabupaten/Kota)

In [11]:
r = requests.get('https://covid19.riau.go.id/webster')
soup = BeautifulSoup(r.content, 'html.parser')

### Untuk ditampilkan di chatbot

In [12]:
print('Rekapitulasi Kasus COVID-19 di Provinsi Riau Per Tanggal:', date.today())
angka = []
for data in soup.find_all('table')[2].find_all('td'):
    data = data.text
    data = int(data)
    angka.append(data)
print("Sedang Dirawat:", angka[0])
print("Sembuh:", angka[1])
print("Meninggal:", angka[2])
print("Total:", angka[0] + angka[1] + angka[2])

Rekapitulasi Kasus COVID-19 di Provinsi Riau Per Tanggal: 2020-07-28
Sedang Dirawat: 105
Sembuh: 266
Meninggal: 11
Total: 382


### Dijadikan Dataset (Update Per Hari)
#### Code:
```{python}
with open('riau-dataset.csv', 'a+') as csv_file:
    fieldnames = ['tanggal', '(ODP) Total', '(ODP) Selesai Pemantauan', '(PDP) Total', '(PDP) Sembuh', '(PDP) Meninggal', '(Positif) Dirawat', '(Positif) Sembuh', '(Positif) Meninggal']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    c = 1
    temp = {}
    writer.writeheader()
    for tabel in soup.find_all('table'):
        temp['tanggal'] = date.today()
        for data in tabel.find_all('td'):
            data = data.text
            data = re.sub("\.", "", data)
            temp[fieldnames[c]] = data
            c += 1
    writer.writerow(temp)
```

In [21]:
df = pd.read_csv('riau-dataset.csv')
df.head()

,tanggal,(ODP) Total,(ODP) Selesai Pemantauan,(PDP) Total,(PDP) Sembuh,(PDP) Meninggal,(Positif) Dirawat,(Positif) Sembuh,(Positif) Meninggal
0,2020-07-28,88234,84584,2657,2247,218,105,266,11
